### Import everything

In [28]:
%load_ext autoreload
%autoreload 2
from run import main
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Run everything

In [2]:
result, data = main()

Start script


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating 

#### Save to pickle

In [90]:
def save_to_pickle(data):
    pickle.dump( data, open( "data.p", "wb" ) )

In [91]:
save_to_pickle((result, data))

In [92]:
first = list(map(lambda x: x[1], result))
first_max = min(first, key=lambda x: x.rmse)

In [93]:
def find_best_models(results):
    min_models = []
    for m in range(len(results[0])):
        bla = []
        for r in results:
            bla.append(r[m])
        min_model = min(bla, key=lambda x: x.rmse)
        min_models.append(min_model)
    return min_models

In [94]:
best_models = find_best_models(result)

In [132]:
models = best_models

## Blend 'em together

In [133]:
def build_predictions_table(data, models):
    blending_data = np.zeros((len(data), len(models)))
    for i_d, x in enumerate(data):
        for i_m, model in enumerate(models):
            pred = model.algo.predict(str(x[0]), str(x[1]))
            blending_data[i_d, i_m] = pred.est
    return blending_data

In [134]:
x = build_predictions_table(data, models)
y = list(map(lambda x: x[2], data))

In [135]:
print(x[:1])
print(y[:1])

[[3.45606229 3.45606229 3.45606229 3.45606229 3.45606229 3.45606229
  3.45606229 3.45606229 3.45606229 3.45606229]]
[4.0]


In [136]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
clf.fit(x, y) 
clf.coef_

array([-3.02922588e-28, -3.02922588e-28, -3.02922588e-28, -3.02922588e-28,
       -3.02922588e-28, -3.02922588e-28, -3.02922588e-28, -3.02922588e-28,
       -3.02922588e-28, -3.02922588e-28])

In [137]:
def blend_them_together(models):
    from sklearn.model_selection import cross_validate
    from sklearn.linear_model import Ridge
    a = Ridge(alpha=1)
    cv_results = cross_validate(a, x, y, cv=3,
                                scoring=('neg_mean_squared_error'),
                                return_train_score=True, return_estimator=True)

    test_scores = cv_results['test_score']
    estimators = cv_results['estimator']
    best = estimators[np.argmax(np.array(test_scores))]
    print(best.coef_)
    return best

In [138]:
blender_model = blend_them_together(models)

[-5.75552916e-27 -5.75552916e-27 -5.75552916e-27 -5.75552916e-27
 -5.75552916e-27 -5.75552916e-27 -5.75552916e-27 -5.75552916e-27
 -5.75552916e-27 -5.75552916e-27]


In [139]:
def predict_using_blending(models, trainer, x):
    pred_sum = 0
    input = np.zeros((len(models), ))
    for i, m in enumerate(models):
        prediction = m.algo.predict(str(x[0]), str(x[1])).est
        input[i]  = prediction
    return trainer.predict([input])

In [140]:
def get_all_predictions(models, blending_model):
    blend_predictions = []
    for each in data:
        blend_pred = predict_using_blending(models, blending_model, each)
        blend_pred_round = int(round(blend_pred[0]))
        blend_predictions.append(blend_pred_round)
    return blend_predictions

In [141]:
blending_pred = get_all_predictions(models, blender_model)
np.sqrt(mean_squared_error(blending_pred, y))

1.1416231003721427